In [169]:
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_addons as tfa
import numpy as np
from sklearn.utils import shuffle

import sys 
sys.path.append('../src/utils/')
sys.path.append('../src/models/')
sys.path.append('../src/visualization/')

from particles import Photon
from center_finder import CenterFinder, masked_loss, masked_loss_seed

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load and prepare data for training from one- and two-particle samples

In [199]:
photon_1 = Photon(n_pcl=1)
photon_2 = Photon(n_pcl=2)

photons = [photon_1, photon_2]

In [200]:
# define the desired model of the seed finder for data pre-processing
seed_paths = {"model_path": "../notebooks/models/seed_finder/run_3/", 
              "weight_path": "../notebooks/models/seed_finder/run_3/weights.27-0.0105.hdf5"}

In [201]:
# combine elements for 1 and 2 photon samples into one dataset
Xtrain, ytrain = ([], [], []), {'center':[], 'energy': [], 'seed': []} 
Xvalid, yvalid = ([], [], []), {'center':[], 'energy': [], 'seed': []} 

for photon in photons:
    Xi, yi = photon.data_for_center_finder("debug", 0.3, **seed_paths)
    [Xtrain[i].extend(Xi[i]) for i in range(3)]
    {ytrain[i].extend(yi[i]) for i in ['center', 'energy', 'seed']}

    Xi, yi = photon.data_for_center_finder("debug", 0.3, **seed_paths)
    [Xvalid[i].extend(Xi[i]) for i in range(3)]
    {yvalid[i].extend(yi[i]) for i in ['center', 'energy', 'seed']}

Xtrain = [np.array(Xtrain[i]) for i in range(3)]
ytrain = {i: np.array(ytrain[i]) for i in ['center', 'energy', 'seed']}

Xvalid = [np.array(Xvalid[i]) for i in range(3)]
yvalid = {i: np.array(yvalid[i]) for i in ['center', 'energy', 'seed']}

100%|██████████| 1000/1000 [00:00<00:00, 2310.10it/s]


1094/1094 [==============================] - 4s 3ms/step


100%|██████████| 1000/1000 [00:00<00:00, 2571.64it/s]


1094/1094 [==============================] - 4s 3ms/step


100%|██████████| 1000/1000 [00:00<00:00, 3859.42it/s]


In [156]:
# shuffle the samples randomly
Xtrain[0], Xtrain[1], Xtrain[2], ytrain["center"], ytrain["energy"], ytrain["seed"] = shuffle(Xtrain[0], Xtrain[1], 
                                                                                              Xtrain[2], ytrain["center"], 
                                                                                              ytrain["energy"], ytrain["seed"], 
                                                                                              random_state=42)
Xvalid[0], Xvalid[1], Xvalid[2], yvalid["center"], yvalid["energy"], yvalid["seed"] = shuffle(Xvalid[0], Xvalid[1], 
                                                                                              Xvalid[2], yvalid["center"], 
                                                                                              yvalid["energy"], yvalid["seed"], 
                                                                                              random_state=42)

# Prepare the model and training

In [160]:
# define parameters of the model
args = {
    'n' : 4,
    'conv_filter' : [32, 64], 
    'conv_kernel' : [3, 3],
    'dense_layers' : [1000, 500, 200, 200],
    'dropout' : [0.1, 0.1, 0.3, 0.1],
    'en_max' : 100,
}

In [195]:
model = CenterFinder(args)
opt = tfa.optimizers.LAMB(learning_rate=0.0001)

loss = {'energy':masked_loss, 'center':masked_loss,
                       'seed': masked_loss_seed}

loss_weights = {'energy':1, 'center': 1, 'seed': 0.05}
model.compile(optimizer=opt, loss=loss, loss_weights=loss_weights)

In [196]:
filepath = 'models/center_finder/run_1/'

checkpoint = ModelCheckpoint(
        str(filepath)+"/weights.{epoch:02d}-{val_loss:.4f}.hdf5", 
        monitor='val_loss', verbose = 1, save_best_only = True, 
        mode='auto', save_freq='epoch')

callback = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 30)

In [197]:
Xtrain[0].shape

(560, 7, 7, 4)

In [188]:
np.where(np.sum(Xtrain[0] != -1, axis=(1,2,3)) == 0 )

(array([566]),)

In [189]:
Xtrain[0] = Xtrain[0][:560]
Xtrain[1] = Xtrain[1][:560]
Xtrain[2] = Xtrain[2][:560]
ytrain["center"] = ytrain["center"][:560]
ytrain["energy"] = ytrain["energy"][:560]
ytrain["seed"] = ytrain["seed"][:560]

# Model training

In [198]:
history = model.fit(Xtrain, ytrain,
                    validation_data=(Xvalid, yvalid), 
                    epochs=30, batch_size=512, 
                    callbacks=[checkpoint, callback])

Epoch 1/30
2/2 [==============================] - ETA: 0s - loss: 0.7386 - center_loss: 0.4135 - energy_loss: 0.3207 - seed_loss: 0.0877 
Epoch 1: val_loss did not improve from inf
2/2 [==============================] - 19s 5s/step - loss: 0.7386 - center_loss: 0.4135 - energy_loss: 0.3207 - seed_loss: 0.0877 - val_loss: nan - val_center_loss: nan - val_energy_loss: nan - val_seed_loss: nan
Epoch 2/30
2/2 [==============================] - ETA: 0s - loss: 0.7248 - center_loss: 0.4008 - energy_loss: 0.3198 - seed_loss: 0.0844
Epoch 2: val_loss did not improve from inf
2/2 [==============================] - 3s 2s/step - loss: 0.7248 - center_loss: 0.4008 - energy_loss: 0.3198 - seed_loss: 0.0844 - val_loss: nan - val_center_loss: nan - val_energy_loss: nan - val_seed_loss: nan
Epoch 3/30


KeyboardInterrupt: 